In [1]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.pretrained import *

import warnings
warnings.filterwarnings("ignore")

from pyspark.context import SparkContext 
from pyspark import SparkConf
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os 
os.environ['PYSPARK_PYTHON'] = "/root/miniconda3/envs/sp/bin/python"

spark = SparkSession.builder\
    .master("local[*]")\
    .config("spark.driver.memory", "16G")\
    .config("spark.driver.maxResultSize", "0")\
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")\
    .config("spark.kryoserializer.buffer.max", "2000m")\
    .config("spark.jsl.settings.pretrained.cache_folder", "sample_data/pretrained")\
    .config("spark.jsl.settings.storage.cluster_tmp_dir", "sample_dataorage")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.4.1")\
    .config(
    "spark.jars",
    "/data/jupyter-data/lab08/kafka-clients-3.4.0.jar,/data/jupyter-data/lab08/spark-sql-kafka-0-10_2.12-3.3.1.jar, \
    /data/jupyter-data/lab08/spark-token-provider-kafka-0-10_2.12-3.3.1.jar, \
    /data/jupyter-data/lab08/commons-pool2-2.11.1.jar") \
    .config("spark.executorEnv.PYSPARK_PYTHON","/root/miniconda3/3/envs/sp/bin/python") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .appName("Pro2").getOrCreate()

:: loading settings :: url = jar:file:/root/miniconda3/envs/sp/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8627fe3b-fefe-4e95-bcac-86b0c138a51e;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.4.1 in central
:: resolution report :: resolve 75ms :: artifacts dl 3ms
	:: modules in use:
	com.johnsnowlabs.nlp#spark-nlp_2.12;4.4.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-8627fe3b-fefe-4e95-bcac-86b0c138a

23/05/19 02:16:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/19 02:16:29 WARN DependencyUtils: Local jar /data/jupyter-data/lab08/kafka-clients-3.4.0.jar does not exist, skipping.
23/05/19 02:16:29 WARN DependencyUtils: Local jar /data/jupyter-data/lab08/spark-sql-kafka-0-10_2.12-3.3.1.jar does not exist, skipping.
23/05/19 02:16:29 WARN DependencyUtils: Local jar /data/jupyter-data/lab08/spark-token-provider-kafka-0-10_2.12-3.3.1.jar does not exist, skipping.
23/05/19 02:16:29 WARN DependencyUtils: Local jar /data/jupyter-data/lab08/commons-pool2-2.11.1.jar does not exist, skipping.
23/05/19 02:16:29 INFO SparkContext: Running Spark version 3.3.1
23/05/19 02:16:29 INFO ResourceUtils: ==============================================================
23/05/19 02:16:29 INFO ResourceUtils: No custom resources configured for spark.driver.
23/05/19 02:16:29 INFO ResourceUtils: ==============================

In [2]:
MODEL_NAME = 'distilbert-base-cased-distilled-squad'

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context", "true_ans"]) \
    .setOutputCols(["document_question", "document_context", "document_ans"])

spanClassifier_loaded = DistilBertForQuestionAnswering.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

Py4JJavaError: An error occurred while calling None.com.johnsnowlabs.nlp.annotators.classifier.dl.DistilBertForQuestionAnswering.
: java.lang.NoClassDefFoundError: com/amazonaws/AmazonServiceException
	at com.johnsnowlabs.nlp.serialization.Feature.<init>(Feature.scala:36)
	at com.johnsnowlabs.nlp.serialization.MapFeature.<init>(Feature.scala:195)
	at com.johnsnowlabs.nlp.annotators.classifier.dl.DistilBertForQuestionAnswering.<init>(DistilBertForQuestionAnswering.scala:149)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:238)

In [3]:
schema = StructType([
    StructField("question", StringType(), True),
    StructField("context", StringType(), True),
    StructField("true_ans", StringType(), True)
])

kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "pro2_13") \
    .load()

json_stream = kafka_stream.select(from_json(kafka_stream.value.cast("string"), schema).alias("data"))
qa_stream = json_stream.select("data.question", "data.context", "data.true_ans")

query = qa_stream.writeStream \
    .outputMode("append") \
    .format("memory") \
    .option("queryName", "QA_model") \
    .trigger(processingTime = '5 seconds')
    
query.start()

23/05/16 06:37:14 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-685b763d-8cdf-40b8-b985-0c8eac9ef807. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/05/16 06:37:14 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/05/16 06:37:15 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


In [4]:
df1 = spark.sql("SELECT * FROM QA_model")
df1.show(5, truncate=False)

+----------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------+
|question                                                        |context                                                                                         

In [9]:
result = pipeline.fit(df1).transform(df1)
df2 = result.select("question", "context", "true_ans", "answer.result")
df2.show(30)

+--------------------+--------------------+--------------------+--------------------+
|            question|             context|            true_ans|              result|
+--------------------+--------------------+--------------------+--------------------+
|In what country i...|The Normans (Norm...|              France|            [France]|
|When were the Nor...|The Normans (Norm...|10th and 11th cen...|[10th and 11th ce...|
|From which countr...|The Normans (Norm...|Denmark, Iceland ...|[Denmark , Icelan...|
|Who was the Norse...|The Normans (Norm...|               Rollo|             [Rollo]|
|What century did ...|The Normans (Norm...|        10th century|              [10th]|
|Who was the duke ...|The Norman dynast...|William the Conqu...|[William the Conq...|
|Who ruled the duc...|The Norman dynast...|           Richard I|         [Richard I]|
|What religion wer...|The Norman dynast...|            Catholic|          [Catholic]|
|What is the origi...|The English name ...|           

In [ ]:
query.stop()
spark.sql("drop view QA_model")